In [ ]:
!pip install --upgrade pip --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers accelerate einops langchain xformers bitsandbytes sentence_transformers chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cuda.is_available()

True

In [ ]:
model_id = os.path.join("/content","drive","My Drive", "cache", "Llama-2-7b-chat-hf", "snapshots", "1")

In [ ]:
os.path.exists(model_id)


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")


Prepare pipeline: 4.405 sec.


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:


test_model(tokenizer,
           query_pipeline,
           "As a fiction author provide a 200 word detailed description of a Russian invasion of Moldova.")



Test inference: 198.876 sec.
Result: As a fiction author provide a 200 word detailed description of a Russian invasion of Moldova.
The sun had just risen over the horizon as the Russian army marched into Moldova. The once peaceful streets were now filled with the sounds of tanks and soldiers, as the Russian forces took control of the country. The Moldovan military, vastly outnumbered and outgunned, put up a weak resistance but were quickly overpowered.
As the Russian forces advanced, they were met with little resistance. The Moldovan government, fearing for their lives and their country's sovereignty, had fled the capital city of Chisinau. The Russian flag now flew high above the city, symbolizing the country's new occupation.
The streets were filled with the sounds of chaos and destruction. Buildings were looted and set ablaze, and the once vibrant city was now a war zone. The Moldovan people, who had once lived in peace and prosperity, were now forced to live under the rule of their 

In [ ]:

test_model(tokenizer,
           query_pipeline,
           "Write me a detailed scenario for a potential Russian invasion of Moldova.")

Test inference: 554.769 sec.
Result: Write me a detailed scenario for a potential Russian invasion of Moldova.
Introduction:
Moldova, a small, landlocked country located between Romania and Ukraine, has been facing a complex geopolitical situation in recent years. The country has been struggling to maintain its independence and sovereignty in the face of external pressures from both Russia and Ukraine. In this scenario, we will explore a potential Russian invasion of Moldova and the consequences that could follow.
Scenario:
It is 2025, and tensions between Russia and Moldova have been escalating for several months. Russia has been accusing Moldova of collaborating with Ukraine and the West, and has been using various means to exert pressure on the Moldovan government. The Moldovan government, on the other hand, has been trying to maintain its independence and sovereignty, but has been struggling to counter Russian aggression.
One day, Russian President Vladimir Putin announces that Rus